In [41]:
# Import additional dependencies
import random
from shapely.geometry import LineString

In [42]:
# Helper functions
def read_data(filename):
    # Read and parse example data files
    with open('./data/' + filename) as f:
        size = list(map(int, f.readline().split(';')))
        coordinates = [list(map(int, i.split(';'))) for i in f.readlines()]
        return size, coordinates


def random_hex_color():
    # Generates random HEX color
    return '#%06X' % random.randint(0, 256 ** 3 - 1)


def line_intersect(x1, y1, x2, y2, x3, y3, x4, y4):
    # Checks if two segments intersect each other.
    return LineString([(x1, y1), (x2, y2)]).intersects(LineString([(x3, y3), (x4, y4)]))

In [43]:
# Constans
SAVE_IMAGES = True
POPULATION_SIZE = 2
MUTATION_CHANCE = 100
EPOCHES = 1
LINE_DIS = 40
MAX_SEGMENTS = 4
MAX_SEGMENT_DIS = 4
COST_FUNCTION_WEIGHTS = [.5, .1, .05, .15, .2]

# Dynamic variables
(field_width, field_height), coordinates = read_data('zad1.txt')

In [44]:
class Individual:
    def __init__(self):
        self.paths = []

    def __str__(self):
        return '\n'.join(str(i) for i in self.paths)

    # todo: add fitness_score


class Path:
    def __init__(self, xs=0, ys=0, xe=0, ye=0):
        self.xs = xs
        self.ys = ys
        self.xe = xe
        self.ye = ye
        self.segments = []
        self.color = random_hex_color()

    def __str__(self):
        return '(%s, %s); (%s, %s);\n' % (self.xs, self.ys, self.xe, self.ye) + '\t'.join(str(i) for i in self.segments)


class Segment:
    def __init__(self, dir='', dis=0):
        self.dir = dir
        self.dis = dis

    def __str__(self):
        return '[%s, %s]' % (self.dir, self.dis)

In [45]:
def generate_random_segments(path):
    segments = []
    x, y = path.xs, path.ys
    previous_dir = None

    for i in range(random.randint(0, MAX_SEGMENTS)):
        if x == path.xe and y == path.ye: return segments

        dirs = [*'UDLR']
        if previous_dir: dirs.remove(previous_dir)
        if previous_dir == 'U': dirs.remove('D')
        if previous_dir == 'D': dirs.remove('U')
        if previous_dir == 'L': dirs.remove('R')
        if previous_dir == 'R': dirs.remove('L')

        dr = random.choice(dirs)
        ds = random.randint(1, MAX_SEGMENT_DIS)
        segment = Segment(dir=dr, dis=ds)
        segments.append(segment)

        if dr == 'U': y -= ds
        if dr == 'D': y += ds
        if dr == 'L': x -= ds
        if dr == 'R': x += ds
        previous_dir = dr

    segments += connect_random_segments(Path(x, y, path.xe, path.ye))
    return segments


def connect_random_segments(path):
    # If they are on the same line
    if path.xs == path.xe:
        if path.ys < path.ye: return [Segment(dir='D', dis=path.ye - path.ys)]
        elif path.ys > path.ye: return [Segment(dir='U', dis=path.ys - path.ye)]
        else: return []

    if path.ys == path.ye:
        if path.xs > path.xe: return [Segment(dir='L', dis=path.xs - path.xe)]
        elif path.xs < path.xe: return [Segment(dir='R', dis=path.xe - path.xs)]
        else: return []

    def horizontal(segment):
        segment.dir = 'R' if path.xs < path.xe else 'L'
        segment.dis = abs(path.xs - path.xe)

    def vertical(segment):
        segment.dir = 'D' if path.ys < path.ye else 'U'
        segment.dis = abs(path.ys - path.ye)

    segments = []
    previous_dir = None
    for _ in range(2):
        segment = Segment()
        if previous_dir in ('U', 'D'): horizontal(segment)
        elif previous_dir in ('L', 'R'): vertical(segment)
        else: 
            if random.random() > 0.5: horizontal(segment)
            else: vertical(segment)
        previous_dir = segment.dir
        segments.append(segment)

    return segments


def initial_population():
    population = []
    for _ in range(POPULATION_SIZE):
        individual = Individual()
        for [x, y, xe, ye] in coordinates:
            path = Path(x, y, xe, ye)
            path.segments =  generate_random_segments(path) # connect_random_segments(path)
            individual.paths.append(path)
        population.append(individual)
    return population


def fitness_score(ind):
    summary_distance = 0
    summary_segments = 0
    crossouts = 0
    summary_distance_outside = 0
    summary_segments_outside = 0
    
    lines = [] # Store all segments eg. [x, y, xe, ye, path]
    for path in ind.paths:
        summary_segments += len(path.segments)

        x, y = path.xs, path.ys
        for seg in path.segments:
            summary_distance += seg.dis 

            xe, ye = x, y
            if seg.dir == 'U': ye -= seg.dis
            if seg.dir == 'D': ye += seg.dis
            if seg.dir == 'L': xe -= seg.dis
            if seg.dir == 'R': xe += seg.dis
            lines.append([x, y, xe, ye, path])

            for i in range(seg.dis):
                if x <= 0 or y <= 0 or x > field_width or y > field_height: summary_distance_outside += 1
                if seg.dir == 'U': y -= 1
                if seg.dir == 'D': y += 1
                if seg.dir == 'L': x -= 1
                if seg.dir == 'R': x += 1
            
            if x <= 0 or x > field_width or y <= 0 or y >= field_height:
                summary_segments_outside += 1
            else:
                if seg.dir == 'D' and y - seg.dis <= 0: summary_segments_outside += 1
                if seg.dir == 'U' and y + seg.dis > field_height: summary_segments_outside += 1
                if seg.dir == 'R' and x - seg.dis <= 0: summary_segments_outside += 1
                if seg.dir == 'L' and x + seg.dis > field_width: summary_segments_outside += 1

    checked = []
    for i in range(len(lines)):
        for j in range(len(lines)):
            if i != j and lines[i][4] != lines[j][4] and [j, i] not in checked and [j, i] not in checked:
                checked.append([i, j])
                crossouts += 1

    summary_distance_outside += 1 if summary_distance_outside != 0 else 0
    return sum([a * b for a, b in zip([crossouts, summary_distance, summary_segments, summary_segments_outside, summary_distance_outside], COST_FUNCTION_WEIGHTS)])


def select(array, n):
    pass


def crossover(parent1, parent2):
    pass


def mutate(ind):
    pass

In [46]:
# Line intersection tests
l1 = [1, 1, -1, 1]
l2 = [-1, 1, -1, 5]
assert line_intersect(*l1, *l2) == True

# Fitness score manual test
test_individual = Individual()

path1 = Path(1, 1, 5, 5)
s1 = Segment('L', 2)
s2 = Segment('D', 4)
s3 = Segment('R', 2)
path1.segments = [s1, s2, s3]
test_individual.paths = [path1]

fitness_score(test_individual)

3.0

In [47]:
population = initial_population()

for _ in range(EPOCHES):
    for ind in population:
        print(fitness_score(ind))

print(population[0])

339.90000000000003
126.4
(2, 7); (9, 7);
[U, 3]	[L, 4]	[D, 3]	[R, 11]
(3, 8); (7, 6);
[U, 4]	[L, 4]	[U, 4]	[R, 8]	[D, 6]
(4, 4); (5, 13);
[D, 2]	[R, 2]	[U, 1]	[R, 1]	[D, 8]	[L, 2]
(5, 2); (10, 12);
[L, 3]	[D, 4]	[R, 8]	[D, 6]
(6, 6); (6, 8);
[D, 1]	[R, 1]	[U, 4]	[L, 2]	[R, 1]	[D, 5]
(7, 10); (13, 10);
[D, 4]	[L, 4]	[D, 4]	[R, 4]	[R, 6]	[U, 8]
(8, 2); (8, 15);
[D, 3]	[D, 10]
(10, 10); (13, 6);
[R, 3]	[U, 2]	[R, 4]	[U, 2]	[L, 4]


In [48]:
from PIL import Image
import tkinter as tk

C_WIDTH = field_width * LINE_DIS
C_HEIGHT = field_height * LINE_DIS

root = tk.Tk()
root.title('Genetic Algorithm')
root.resizable(False, False)

w = tk.Canvas(root, width=C_WIDTH, height=C_HEIGHT, bg="#505050")
w.pack()
root.update()


def draw_individual(ind):
    # Draw individual by drawing all segments for ecah path
    offset = LINE_DIS / 2
    for path in ind.paths:
        x, y = path.xs * LINE_DIS, path.ys * LINE_DIS
        for segment in path.segments:
            x_gap, y_gap = 0, 0
            if segment.dir in ('U', 'D'): y_gap = LINE_DIS
            if segment.dir in ('L', 'R'): x_gap = LINE_DIS
            
            x_sign, y_sign = 1, 1
            if segment.dir == 'U': y_sign = -1
            if segment.dir == 'L': x_sign = -1

            for i in range(segment.dis):
                w.create_line(x - offset, y - offset, x + x_gap * x_sign - offset, y - offset + y_sign * y_gap, fill=path.color) # fill='#aaa'
                if segment.dir == 'U': y -= LINE_DIS
                if segment.dir == 'D': y += LINE_DIS
                if segment.dir == 'L': x -= LINE_DIS
                if segment.dir == 'R': x += LINE_DIS


def draw_field():
    # Draw background dots
    for x in range(0, C_WIDTH, LINE_DIS):
        for y in range(0, C_HEIGHT, LINE_DIS):
            draw_rect(w, x, y, x + LINE_DIS, y + LINE_DIS, _offset=2)


def draw_rect(w, x1, y1, x2, y2, _offset=2, color='#bbb'):
    # Helper function to draw a circle
    offset = LINE_DIS / _offset
    w.create_oval(x1 + offset, y1 + offset, x2 - offset, y2 - offset, fill=color, outline='')


def draw_coords():
    # Draw points by their coordinates
    for x in coordinates:
        color = random_hex_color()
        x = list(map(lambda e: (e - 1) * LINE_DIS, x))
        draw_rect(w, x[0], x[1], x[0] + LINE_DIS, x[1] + LINE_DIS, _offset=3, color=color)
        draw_rect(w, x[2], x[3], x[2] + LINE_DIS, x[3] + LINE_DIS, _offset=3, color=color)


for i, ind in enumerate(population):
    w.delete('all')

    draw_field()
    draw_individual(ind)
    draw_coords()

    if SAVE_IMAGES:
        # TODO: Save by making screen
        filename = f"models/{i}_ind"
        w.postscript(file=filename + '.ps', colormode='color')
        img = Image.open(filename + '.ps')
        img.save(filename + '.png', "png")

root.mainloop()